In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import re 

#initialize webdriver
def start_driver():
    driver = webdriver.Chrome()
    driver.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31')
    time.sleep(5)
    return driver

#This function for scrap movies data
def scrape_genre_data(genre):
    driver = start_driver()

    #This for scroll and find genre button
    scroll_attempts = 0
    while scroll_attempts < 5:
        driver.execute_script("window.scrollBy(0, 100);")
        time.sleep(1)

        try:
            genre_section = driver.find_element(By.XPATH, "//div[contains(text(), 'Genre')]")
            genre_section.click()
            time.sleep(2)
            break  #stop scrolling button found means
        except:
            scroll_attempts += 1
            continue

    #scroll and find genre name button
    scroll_attempts = 0
    while scroll_attempts < 3:
        driver.execute_script("window.scrollBy(0, 100);")
        time.sleep(1.5)
        try:
            print(f"Selecting genre: {genre}")
            genre_button = driver.find_element(By.XPATH, f"//button[span[contains(text(), '{genre}')]]")
            genre_button.click()
            time.sleep(2)
            break
        except:
            print(f"{genre} genre not found")
            scroll_attempts += 1
            continue
    # This loop for scrap each movie data without duplicates  
    movies = []
    while len(movies) < 500: #this will stop total count meet

        #This loop for each page having 50 movies and it will stop after getting 50 movies
        new_movies = 0
        while new_movies < 50 and len(movies) < 500:
            movie_elements = driver.find_elements(By.XPATH, "//a[h3[@class='ipc-title__text ipc-title__text--reduced']]") 
            for movie in movie_elements:
                movie_name = movie.text.strip()

                #Here I used regular expression to remove digits,symbol and extra spaces in movie name
                movie_name = re.sub(r'^\d+\.\s*', '', movie_name) 
                if movie_name and movie_name not in [m['Movie Name'] for m in movies]: #This will store movie names not in a list
                    try:
                        raw_duration = movie.find_element(By.XPATH, "(./ancestor::li//span[contains(@class, 'dli-title-metadata-item')])[2]").text.strip()
                        #This for convert raw duration to minutes for other calculation apply
                        movie_duration = convert_duration(raw_duration) 
                    except:
                        movie_duration = "N/A"

                    # For rating
                    try:
                        movie_rate = movie.find_element(By.XPATH, "./ancestor::li//span[contains(@class, 'ipc-rating-star--rating')]").text.strip()
                        movie_rating = float(movie_rate)
                    except:
                        movie_rating = "N/A"
                    # For votes
                    try:
                        movie_votes = movie.find_element(By.XPATH, "./ancestor::li//span[contains(@class, 'ipc-rating-star--voteCount')]").text.strip()
                        # From votes string like this Ex.(10k) so that regular expression is needed
                        movie_votes = re.sub(r"[()]", "", movie_votes)
                        if 'K' in movie_votes:
                            movie_votes = int(float(movie_votes.replace('K', '')) * 1000)
                        elif 'M' in movie_votes:
                            movie_votes = int(float(movie_votes.replace('M', '')) * 1000000)
                        else:
                            movie_votes = int(movie_votes)
                    except:
                        movie_votes = "N/A"

                    movies.append({
                        "Genre": genre,
                        "Movie Name": movie_name,
                        "Duration": raw_duration,
                        "Duration_Minutes": movie_duration,
                        "Rating": movie_rating,
                        "Votes": movie_votes
                    })
                    new_movies += 1
                    print({movie_name,raw_duration,movie_duration,movie_rating,movie_votes})

                    #scroll each movie collected
                    driver.execute_script("window.scrollBy(0, 200);")
                    time.sleep(1)


        #scroll to find a load more button
        scroll_up_attempts = 0
        while scroll_up_attempts < 5 and len(movies) < 500:
            driver.execute_script("window.scrollBy(0, -100);")
            time.sleep(1)
            try:
                load_more_button = driver.find_element(By.XPATH, "//*[@id='__next']/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button")
                load_more_button.click()
                time.sleep(2)
                break
            except:
                scroll_up_attempts += 1
                continue

    #scroll for back to top button
    scroll_up_attempts = 0
    while len(movies) >= 500 and scroll_up_attempts < 5:
        try:
            back_to_top = driver.find_element(By.XPATH, "//*[@id='__next']/button")
            back_to_top.click()
            time.sleep(3)
            break
        except:
            scroll_up_attempts += 1
            continue

    driver.quit()

    # To save as .csv
    df = pd.DataFrame(movies)
    df.to_csv(f"{genre.lower()}_movies.csv", index=False)
    print(f"Data saved as {genre}_movies.csv")


#convert duration function
def convert_duration(duration_text):
    hours = 0
    minutes = 0
    # Here Regular expression doing for covert as total minutes Ex. 1h 30m to 90 minutes
    match = re.match(r"(?:(\d+)h)?\s*(?:(\d+)m)?", duration_text)
    if match:
        if match.group(1): # If there's an hour like "1h"
            hours = int(match.group(1)) * 60 
        if match.group(2):  # If there's a minute like "30m"
            minutes = int(match.group(2))
    return hours + minutes

selected_genre = "Action"  #change genre name here to collect next genre data
scrape_genre_data(selected_genre)


In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text

# Database connection 
host = "gateway01.eu-central-1.prod.aws.tidbcloud.com"
port = 4000
user = "42aq8sKC2dkkKnC.root"
password = "YOUR PASSWORD"
ssl_args = "?ssl_ca=/etc/ssl/certs/ca-certificates.crt"

# Temporary engine to create database
temp_engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/information_schema{ssl_args}"
)

# Create grab database
with temp_engine.connect() as conn:
    conn.execute(text("CREATE DATABASE IF NOT EXISTS grab"))

# Connect to the grab database
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/grab{ssl_args}"
)

# Create movies table
create_table_sql = """
CREATE TABLE IF NOT EXISTS movies (
    genre VARCHAR(255),
    movie_name VARCHAR(255),
    duration VARCHAR(100),
    duration_minutes INT,
    rating FLOAT,
    votes INT
)
"""
with engine.connect() as conn:
    conn.execute(text(create_table_sql))

# Load CSV
csv_path = "E:\Sakthi\prasanth\projects\imdb\Scripts\all_genres.csv"  
df = pd.read_csv(csv_path)

# Insert into movies table
df.to_sql(
    name="movies",
    con=engine,
    if_exists="append",
    index=False
)

print("Movies table created and data uploaded successfully.")
